<a href="https://colab.research.google.com/github/jsj9185/kaggle/blob/main/Titanic(22_12_30).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Data Loading & Preprocessing

In [ ]:
import pandas as pd
from sklearn import linear_model
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/titanic/train.csv')
test = pd.read_csv('/content/drive/MyDrive/titanic/test.csv')

In [ ]:
X_train = train.drop(['Survived'], axis=1) #X,y 분류
y_train = train['Survived']

X_test = test

In [ ]:
X_train = X_train.drop(['PassengerId', 'Ticket', 'Name', 'Cabin'], axis=1) #cabin 결측치가 3분의2고 별 의미 없어보임, name이랑 ticket도
X_test = X_test.drop(['PassengerId', 'Ticket', 'Name', 'Cabin'], axis=1)

In [ ]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,S
887,1,female,19.0,0,0,30.0000,S
888,3,female,NaN,1,2,23.4500,S
889,1,male,26.0,0,0,30.0000,C


In [ ]:
X_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...
413,3,male,NaN,0,0,8.0500,S
414,1,female,39.0,0,0,108.9000,C
415,3,male,38.5,0,0,7.2500,S
416,3,male,NaN,0,0,8.0500,S


In [ ]:
X_train.isnull().sum() #Age는 꽤중요해보여서, 회귀로 결측치 채우기로 결정

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [ ]:
X_test[X_test['Fare'].isnull()]

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
152,3,male,60.5,0,0,NaN,S


In [ ]:
X_train[X_train["Pclass"]==3].mean()

<ipython-input-171-da3b881456cf>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_train[X_train["Pclass"]==3].mean()


Pclass     3.000000
Age       25.140620
SibSp      0.615071
Parch      0.393075
Fare      13.675550
dtype: float64

In [ ]:
X_train[X_train['Embarked'].isnull()]

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
61,1,female,38.0,0,0,80.0,NaN
829,1,female,62.0,0,0,80.0,NaN


In [ ]:
train.loc[61, :]

PassengerId                     62
Survived                         1
Pclass                           1
Name           Icard, Miss. Amelie
Sex                         female
Age                           38.0
SibSp                            0
Parch                            0
Ticket                      113572
Fare                          80.0
Cabin                          B28
Embarked                       NaN
Name: 61, dtype: object

In [ ]:
X_train = X_train.dropna(subset=['Embarked']) #embarked에 있는 결측치 2개 제거
y_train = y_train.drop([61, 829])

In [ ]:
len(y_train)

889

In [ ]:
X_test.loc[152,'Fare'] = 13.68
X_test.isnull().sum()

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         0
Embarked     0
dtype: int64

In [ ]:
X_train = pd.get_dummies(data = X_train, columns=['Embarked'])
X_train['Sex'] = X_train['Sex'].map({'female': 0, 'male': 1}) #0 또는 1로 map

X_test = pd.get_dummies(data = X_test, columns=['Embarked'])
X_test['Sex'] = X_test['Sex'].map({'female': 0, 'male': 1})

In [ ]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,3,1,22.0,1,0,7.2500,0,0,1
1,1,0,38.0,1,0,71.2833,1,0,0
2,3,0,26.0,0,0,7.9250,0,0,1
3,1,0,35.0,1,0,53.1000,0,0,1
4,3,1,35.0,0,0,8.0500,0,0,1
...,...,...,...,...,...,...,...,...,...
886,2,1,27.0,0,0,13.0000,0,0,1
887,1,0,19.0,0,0,30.0000,0,0,1
888,3,0,NaN,1,2,23.4500,0,0,1
889,1,1,26.0,0,0,30.0000,1,0,0


In [ ]:
X_train.isnull().sum() #age 177개 채워주기

Pclass          0
Sex             0
Age           177
SibSp           0
Parch           0
Fare            0
Embarked_C      0
Embarked_Q      0
Embarked_S      0
dtype: int64

In [ ]:
lin_reg = linear_model.LinearRegression() #결측치 채우기 위해 LR 모델 생성

X = X_train.dropna().drop(['Age'], axis=1)
y = X_train.dropna()['Age'] 

lin_reg_model = lin_reg.fit(X, y) #결측치 제거된 걸로만 학습

In [ ]:
y_pred_train = lin_reg_model.predict(X_train.drop(['Age'], axis=1))
X_train['Age'].fillna(pd.Series(y_pred_train.flatten()), inplace=True)

y_pred_test = lin_reg_model.predict(X_test.drop(['Age'], axis=1))
X_test['Age'].fillna(pd.Series(y_pred_test.flatten()), inplace=True)

In [ ]:
X_train.isnull().sum(), X_test.isnull().sum()

(Pclass        0
 Sex           0
 Age           0
 SibSp         0
 Parch         0
 Fare          0
 Embarked_C    0
 Embarked_Q    0
 Embarked_S    0
 dtype: int64, Pclass        0
 Sex           0
 Age           0
 SibSp         0
 Parch         0
 Fare          0
 Embarked_C    0
 Embarked_Q    0
 Embarked_S    0
 dtype: int64)

In [ ]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,3,1,22.00000,1,0,7.2500,0,0,1
1,1,0,38.00000,1,0,71.2833,1,0,0
2,3,0,26.00000,0,0,7.9250,0,0,1
3,1,0,35.00000,1,0,53.1000,0,0,1
4,3,1,35.00000,0,0,8.0500,0,0,1
...,...,...,...,...,...,...,...,...,...
886,2,1,27.00000,0,0,13.0000,0,0,1
887,1,0,19.00000,0,0,30.0000,0,0,1
888,3,0,31.31522,1,2,23.4500,0,0,1
889,1,1,26.00000,0,0,30.0000,1,0,0


# 2. Create & Train Model

In [ ]:
xgb = XGBClassifier(random_state=17)
xgb.fit(X_train, y_train)

XGBClassifier(random_state=17)

In [ ]:
y_result = xgb.predict(X_train)
accuracy_score(y_train, y_result)

0.8785151856017998

In [ ]:
y_pred_xgb = xgb.predict(X_test)
pid = list(range(892, 892+len(X_test)))

In [ ]:
xgb_results = pd.DataFrame({'PassengerId':pid, 'Survived':y_pred_xgb})

In [ ]:
xgb_results.to_csv('submission.csv', index=False) #0.7727

In [ ]:
rf = RandomForestClassifier(random_state=17)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=17)

In [ ]:
y_result = rf.predict(X_train)
accuracy_score(y_train, y_result)

0.9876265466816648

In [ ]:
y_pred_rf = rf.predict(X_test)

In [ ]:
rf_results = pd.DataFrame({'PassengerId':pid, 'Survived':y_pred_rf})
rf_results.to_csv('submission.csv', index=False) #0.7440